In [2]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Config
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 3
EPOCHS = 10

train_dir = '../data/output_dataset/train'
val_dir = '../data/output_dataset/val'

# Data generators
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_data = val_gen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

# Base model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

# Add classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
os.makedirs('models', exist_ok=True)
checkpoint = ModelCheckpoint('vgg16.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
model.fit(train_data, validation_data=val_data, epochs=EPOCHS, callbacks=[checkpoint, earlystop])


Found 259 images belonging to 3 classes.
Found 78 images belonging to 3 classes.
Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 8.3466 - accuracy: 0.2394 
Epoch 1: val_accuracy improved from -inf to 0.29487, saving model to vgg16.h5
9/9 [==============================] - 39s 4s/step - loss: 8.3466 - accuracy: 0.2394 - val_loss: 5.3370 - val_accuracy: 0.2949
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 7.8029 - accuracy: 0.2973 
Epoch 2: val_accuracy did not improve from 0.29487
9/9 [==============================] - 39s 4s/step - loss: 7.8029 - accuracy: 0.2973 - val_loss: 5.0072 - val_accuracy: 0.2821
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 7.3115 - accuracy: 0.2780 
Epoch 3: val_accuracy did not improve from 0.29487
9/9 [==============================] - 39s 5s/step - loss: 7.3115 - accuracy: 0.2780 - val_loss: 4.6470 - val_accuracy: 0.2949
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 6.6055 - accu

In [3]:
test_dir = '../data/output_dataset/test'
test_data = val_gen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=1, class_mode='categorical', shuffle=False)

loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc:.2%}")


Found 78 images belonging to 3 classes.
78/78 [==============================] - 13s 171ms/step - loss: 2.8778 - accuracy: 0.3333
Test Accuracy: 33.33%
